
# Regressão Linear — Capítulo 2 (Notas de Aula)  
Implementação prática em Jupyter Notebook

**Objetivo:** Este notebook implementa e demonstra, no *dataset* anexado, os tópicos do Capítulo 2 das notas de aula:
1. **Regressão linear analítica** (Seção 2.2.1) — solução pela equação normal.  
2. **Regressão linear iterativa com BGD** (Seção 2.2.2) — *Batch Gradient Descent*.  
3. **Regressão linear iterativa com SGD** — *Stochastic Gradient Descent*.  
4. **Avaliação da qualidade da regressão** (Seção 2.3) — métricas (MSE, RMSE, MAE, R²), análise de resíduos e gráficos.

> Obs.: Para a etapa com gradiente, padronizamos as *features* (média 0, desvio-padrão 1) para melhorar a estabilidade numérica e a convergência.


## 1) Carregando os Dataset

In [1]:
from pathlib import Path
import pandas as pd

# A função pd.read_csv() lê o arquivo e o carrega em um DataFrame do pandas.
DATA_PATH = Path.cwd().joinpath(Path("Exercicio01/dataset/data_0460_5832.csv"))
df = pd.read_csv(DATA_PATH)

# O comando .head() mostra as 5 primeiras linhas do DataFrame.
print("Primeiras 5 linhas do dataset:")
print(df.head())

# O comando .info() mostra um resumo técnico, incluindo os tipos de cada coluna.
print('\nInformações do DataFrame:')
df.info()



Primeiras 5 linhas do dataset:
      Sex  Age  Height  Weight  Shoe number
0  Female   53     154      59           36
1    Male   23     170      56           40
2  Female   23     167      63           37
3    Male   21     178      78           40
4  Female   25     153      58           36

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sex          257 non-null    object
 1   Age          257 non-null    int64 
 2   Height       257 non-null    int64 
 3   Weight       257 non-null    int64 
 4   Shoe number  257 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 10.2+ KB


## 2) Tratamento dos Dados

Antes de treinar nosso modelo, precisamos preparar os dados. Isso envolve:

1.  **Lidar com variáveis categóricas**: A coluna `Sex` é categórica ('Male'/'Female'). Os modelos de regressão linear requerem entradas numéricas. Usaremos uma técnica chamada *one-hot encoding* para converter essa coluna em colunas numéricas (0s e 1s).
2.  **Definir Features (X) e Target (y)**: Separaremos nosso conjunto de dados em:
    * `X`: a matriz de features (as variáveis que usaremos para fazer a previsão).
    * `y`: o vetor alvo (a variável que queremos prever, neste caso, `Weight`).

In [12]:

import numpy as np

# Copiar o df original para não sobrescrever
df_encoded = df.copy()

# Identificar colunas categóricas que só têm dois valores
for col in df_encoded.select_dtypes(include=['object', 'category']).columns:
    valores = df_encoded[col].dropna().unique()
    if len(valores) == 2:
        # mapear automaticamente para 0/1
        mapping = {valores[0]: 0, valores[1]: 1}
        df_encoded[col] = df_encoded[col].map(mapping)
        print(f"Coluna {col} mapeada para {mapping}")
    else:
        df_encoded.drop(columns=[col], inplace=True)
# remove colunas constantes
num_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if df_encoded[c].nunique(dropna=True) > 1]

Coluna Sex mapeada para {'Female': 0, 'Male': 1}


## 3) Regressão Linear Iterativa — **Stochastic Gradient Descent (SGD)**

Atualizamos os pesos a **cada exemplo** (com *shuffle* por época), o que pode acelerar a convergência inicial em bases maiores.

**Regra (para amostra $i$):**
$$ \mathbf{w} \leftarrow \mathbf{w} - \eta \; \nabla L_i(\mathbf{w}) = \mathbf{w} - \eta\, (\hat{y}_i - y_i)\, \mathbf{x}_i $$
(com $\mathbf{x}_i$ incluindo o intercepto; aqui usamos *features* padronizadas para estabilidade).


In [13]:
# ===== Regressão linear 1D com SGD (com intercepto) =====
import time, math, os
import numpy as np

def _standardize_xy(x, y):
    """Padroniza x e y e retorna (x_, y_, mu_x, std_x, mu_y, std_y)."""
    mu_x, std_x = float(np.mean(x)), float(np.std(x)) or 1.0
    mu_y, std_y = float(np.mean(y)), float(np.std(y)) or 1.0
    x_ = (x - mu_x) / std_x
    y_ = (y - mu_y) / std_y
    return x_, y_, mu_x, std_x, mu_y, std_y

def _destandardize_w(w0n, w1n, mu_x, std_x, mu_y, std_y):
    """
    Converte os pesos aprendidos no espaço normalizado (y' = w0n + w1n*x')
    para o espaço original (y = w0 + w1*x).
    """
    # y = μy + σy * (w0n + w1n * (x-μx)/σx) = (μy + σy*w0n - σy*w1n*μx/σx) + (σy*w1n/σx) * x
    w1 = (std_y * w1n) / std_x
    w0 = mu_y + std_y*w0n - w1*mu_x
    return float(w0), float(w1)

def _quality_metrics(y_true, y_pred):
    err = y_true - y_pred
    sse = float(np.sum(err**2))
    sst = float(np.sum((y_true - np.mean(y_true))**2))
    ssr = sst - sse
    r2  = 0.0 if sst == 0 else 1.0 - sse/sst
    mse = float(np.mean(err**2))
    return sse, sst, ssr, r2, mse

def fit_line_sgd(x_np, y_np, *,
                 lr=1e-2, epochs=5000, tol=1e-8,
                 normalize=True, batch_size=1,
                 seed=0, patience=200, min_epochs=200):
    """
    SGD 1D com intercepto.
    - normalize: padroniza x e y por par (recomendado p/ SGD).
    - batch_size=1 => SGD puro; ajuste se quiser mini-batch.
    - early stopping: para se melhora < tol durante 'patience' epochs após 'min_epochs'.
    """
    rng = np.random.RandomState(seed)

    # (1) normalização opcional
    if normalize:
        x, y, mu_x, std_x, mu_y, std_y = _standardize_xy(x_np, y_np)
    else:
        x, y = x_np.astype(float), y_np.astype(float)
        mu_x = std_x = mu_y = std_y = None

    n = x.shape[0]
    # adiciona coluna de 1 para intercepto: y ~ w0 + w1*x  =>  y ~ [1, x]·w
    X = np.c_[np.ones(n), x]

    # (2) inicialização
    w = rng.normal(scale=0.01, size=2)  # [w0, w1] no espaço normalizado (se normalize=True)
    best_loss = math.inf
    best_w = w.copy()
    no_improve = 0

    start = time.time()
    loss_hist = []

    # (3) SGD
    for epoch in range(1, epochs+1):
        # permutar amostras a cada época
        idx = rng.permutation(n)
        Xep, yep = X[idx], y[idx]

        # varrer em mini-batches
        for i in range(0, n, batch_size):
            xb = Xep[i:i+batch_size]
            yb = yep[i:i+batch_size]
            # predição e gradiente (MSE)
            pred = xb @ w
            err  = pred - yb
            grad = 2.0/xb.shape[0] * (xb.T @ err)  # ∂/∂w MSE

            # passo SGD
            w -= lr * grad

        # valida perda na época
        pred_full = X @ w
        loss = float(np.mean((pred_full - y)**2))
        loss_hist.append(loss) 
        if loss + tol < best_loss:
            best_loss = loss
            best_w = w.copy()
            no_improve = 0
        else:
            no_improve += 1

        if epoch >= min_epochs and no_improve >= patience:
            break

    wall = time.time() - start
    w = best_w
    # gradiente final (para registro)
    gfinal = 2.0/n * (X.T @ (X @ w - y))
    grad_norm = float(np.linalg.norm(gfinal))

    # (4) métricas no espaço ORIGINAL
    if normalize:
        w0, w1 = _destandardize_w(w[0], w[1], mu_x, std_x, mu_y, std_y)
        y_hat = w0 + w1 * x_np
    else:
        w0, w1 = float(w[0]), float(w[1])
        y_hat = w0 + w1 * x_np

    sse, sst, ssr, r2, mse = _quality_metrics(y_np, y_hat)

    return {
        "w0": w0, "w1": w1,
        "SSE": sse, "SST": sst, "SSR": ssr,
        "R2": r2, "MSE": mse,
        "iters": epoch, "grad_norm": grad_norm,
        "wall_s": wall,
        "kind": "SGD", "N": len(x_np),
        "history": loss_hist 
    }

## 4) Execução da regressão com paralelização

In [14]:
from functools import partial

def run_one_pair(df_encoded, x_col, y_col, *,
                 lr=1e-2, epochs=5000, tol=1e-8,
                 normalize=True, batch_size=1,
                 seed=0):
    pair = df_encoded[[x_col, y_col]].dropna()
    # evita pares degenerados
    if pair[x_col].nunique() < 2 or pair[y_col].nunique() < 2:
        return None

    x_np = pair[x_col].to_numpy(dtype=float)
    y_np = pair[y_col].to_numpy(dtype=float)

    out = fit_line_sgd(
        x_np, y_np,
        lr=lr, epochs=epochs, tol=tol,
        normalize=normalize, batch_size=batch_size,
        seed=seed + hash((x_col, y_col)) % 1000003,  # semente estável por par
        patience=200, min_epochs=200
    )
    out.update({"x": x_col, "y": y_col})
    return out

In [20]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import itertools
import pandas as pd
from pathlib import Path
import os

# Parâmetros do SGD
SGD_CFG = dict(
    lr=1e-2,
    epochs=20000,     # pode reduzir se já convergir rápido
    tol=1e-8,
    normalize=True,
    batch_size=1,
)

cols = df_encoded.columns.tolist()
pairs = list(itertools.permutations(cols, 2))  # pares ordenados (x,y)

def _worker(args):
    x_col, y_col = args
    return run_one_pair(df_encoded, x_col, y_col, **SGD_CFG)

results = []
max_workers = max(1, min(os.cpu_count(), 12))  # limite amigável
print(f"Executando SGD em paralelo com {max_workers} processos…")

with ProcessPoolExecutor(max_workers=max_workers) as ex:
    futs = [ex.submit(_worker, p) for p in pairs]
    for f in as_completed(futs):
        r = f.result()
        if r is not None:
            results.append(r)


Executando SGD em paralelo com 12 processos…


## 5) Análise da qualidade da regressão


Métricas principais:
- **MSE**, **RMSE**, **MAE**
- **Coeficiente de determinação (R²)**: $$ R^2 = 1 - \frac{\sum (y - \hat{y})^2}{\sum (y - \bar{y})^2} $$

Análises gráficas:
- **Resíduos vs. predições** (busca padrão sem estrutura — homocedasticidade)
- **Histograma de resíduos** (aprox. simétrico, média ~ 0)


In [16]:
# DataFrame ordenado com as colunas no formato pedido
order = ["x","y","w0","w1","R2","MSE","SSE","SSR","SST","iters","grad_norm","wall_s","kind","N"]
results_sgd = pd.DataFrame(results)[order].sort_values(["x","y"]).reset_index(drop=True)
display(results_sgd)
print("Total linhas:", len(results_sgd))

,x,y,w0,w1,R2,MSE,SSE,SSR,SST,iters,grad_norm,wall_s,kind,N
0,Age,Height,169.355659,0.025695,0.000850,208.453382,53572.519114,45.550925,53618.070039,330,0.022382,0.627712,SGD,257
1,Age,Sex,0.899398,-0.007457,0.046257,0.205856,52.904908,2.565909,55.470817,248,0.010219,0.485647,SGD,257
2,Age,Shoe number,39.685425,-0.008061,0.001151,9.017788,2317.571551,2.669695,2320.241245,292,0.025525,0.516841,SGD,257
3,Age,Weight,64.048643,0.222637,0.034717,250.637130,64413.742404,2316.646701,66730.389105,310,0.005783,0.640391,SGD,257
4,Height,Age,25.537397,0.017488,0.000819,176.330748,45317.002253,37.130042,45354.132296,377,0.024989,0.707577,SGD,257
5,Height,Sex,-1.967563,0.015561,0.240924,0.163839,42.106575,13.364242,55.470817,465,0.021486,1.300528,SGD,257
6,Height,Shoe number,10.028139,0.173213,0.710838,2.610605,670.925428,1649.315818,2320.241245,414,0.023590,0.775777,SGD,257
7,Height,Weight,-37.169233,0.632686,0.328511,174.353006,44808.722450,21921.666655,66730.389105,488,0.021315,0.884882,SGD,257
8,Sex,Age,32.621235,-6.007016,0.046253,168.312760,43256.379258,2097.753038,45354.132296,447,0.011007,0.839852,SGD,257
9,Sex,Height,159.892881,15.222442,0.240978,158.355227,40697.293446,12920.776593,53618.070039,203,0.015649,0.409146,SGD,257


Total linhas: 20


## 5) Análise da Convergência

In [17]:
import matplotlib.pyplot as plt
import numpy as np

def plot_histories(results_df, n=5, smooth_k=5, max_epochs=None):
    """
    Plota curvas de loss para até n pares armazenados em results_df.
    
    results_df: DataFrame com colunas ["x","y","history",...]
    n: número máximo de pares a plotar
    smooth_k: janela da média móvel para suavizar a curva (se >1)
    max_epochs: número máximo de épocas a exibir (None = usa tudo)
    """
    def moving_average(arr, k):
        if k <= 1 or len(arr) < k:
            return np.array(arr, dtype=float)
        return np.convolve(arr, np.ones(k)/k, mode="valid")

    for i, row in results_df.iterrows():
        if i >= n:
            break
        hist = row["history"]
        if max_epochs is not None:
            hist = hist[:max_epochs]

        x_label = row["x"]
        y_label = row["y"]

        plt.figure(figsize=(10,4))

        # curva bruta
        plt.subplot(1,2,1)
        plt.plot(hist, color="tab:blue")
        plt.title(f"Loss (MSE) - {x_label}→{y_label}")
        plt.xlabel("Época")
        plt.ylabel("Loss")
        plt.grid(alpha=0.3)

        # curva suavizada
        plt.subplot(1,2,2)
        plt.plot(moving_average(hist, smooth_k), color="tab:orange")
        plt.title(f"Loss suavizada ({x_label}→{y_label})")
        plt.xlabel("Época")
        plt.ylabel("Loss (média móvel)")
        plt.grid(alpha=0.3)

        plt.tight_layout()
        plt.show()

In [18]:
# plota só as max_epochs primeiras épocas de até n pares
plot_histories(results_sgd, n=3, smooth_k=0, max_epochs=300)

KeyError: 'history'

## 6) Salvando os dados

In [19]:
from pathlib import Path

# Caminho da pasta onde vamos salvar
DATA_PATH = Path.cwd().joinpath("Exercicio01/tabela")
DATA_PATH.mkdir(parents=True, exist_ok=True)  # cria se não existir

# Ordem desejada das colunas
col_order = [
    "kind", "x", "y", "w0", "w1",
    "SSE", "SSR", "SST", "R2", "MSE",
    "iters", "grad_norm", "wall_s", "peak_mb", "N"
]

# Reorganiza (somente as que realmente existem no DataFrame)
cols_present = [c for c in col_order if c in results_sgd.columns]
results_sgd = results_sgd[cols_present]

# Salva
results_file = DATA_PATH / "results_SGD.csv"
results_sgd.to_csv(results_file, index=False)

print(f"Arquivo salvo em: {results_file}")
print("Colunas na ordem:", results_sgd.columns.tolist())

Arquivo salvo em: /home/nara/MAC5921-Deep-Learning/Exercicio01/tabela/results_SGD.csv
Colunas na ordem: ['kind', 'x', 'y', 'w0', 'w1', 'SSE', 'SSR', 'SST', 'R2', 'MSE', 'iters', 'grad_norm', 'wall_s', 'N']
